# Парсинг банковских рейтингов с сайта banki.ru / Parsing bank ratings from site banki.ru

In [51]:
import requests
import re
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd


### Так выглядит  главная ссылка, в которой мы будем изменять номер страницы / It's main reference, in which  we will change number of page: 

In [52]:
main_url = "http://www.banki.ru/banks/ratings/agency/?PAGEN_2={}&mode=2"

In [53]:
page = 1

In [54]:
requests.get(main_url.format(page))

<Response [200]>

200 ответ --> соединение установлено и данные получены, всё чудесно! **/** 200 answer --> the connection is set and the data is got, good! 

Создадим переменную, которая будет держать HTML код нашей полученной страницы **/** Create var, which will keep HTML-code of got page.

In [55]:
ratings_page = requests.get(main_url.format(page))
ratings_page = ratings_page.content

In [ ]:
ratings_page

Немного структуируем весь этот ужас / Structure this horror :

In [57]:
ratings_page = BeautifulSoup(ratings_page, "lxml")

In [ ]:
ratings_page

## Зная блок  элемента на странице, мы можем находить его в этой куче html-кода / Knowing the block of element on the page, we can find it in this heap of html-code.

**Извлечём номера лицензий и соответсвующие им названия банков / Extract license numbers and corresponding them bank names.**

In [59]:
print(ratings_page.find('div', attrs = {'class':"bank-info"}))

<div class="bank-info">No.2306, Москва и обл.</div>


In [60]:
for num_lic in ratings_page.findAll('div', attrs = {'class':"bank-info"}):
    print(num_lic)

<div class="bank-info">No.2306, Москва и обл.</div>
<div class="bank-info">No.2879, Москва и обл.</div>
<div class="bank-info">No.415, Татарстан</div>
<div class="bank-info">No.1455, Татарстан</div>
<div class="bank-info">No.2880, Москва и обл.</div>
<div class="bank-info">No.2860, Саратовская обл.</div>
<div class="bank-info">No.1459, Москва и обл.</div>
<div class="bank-info">No.1810, Амурская обл.</div>
<div class="bank-info">No.2590, Татарстан</div>
<div class="bank-info">No.2587, Татарстан</div>
<div class="bank-info">No.2529, Мордовия</div>
<div class="bank-info">No.3006, Самарская обл.</div>
<div class="bank-info">No.567, Новосибирская обл.</div>
<div class="bank-info">No.53, Санкт-Петербург и обл.</div>
<div class="bank-info">No.2119, Москва и обл.</div>
<div class="bank-info">No.2602, Саха (Якутия)</div>
<div class="bank-info">No.2388, Алтайский край</div>
<div class="bank-info">No.2659, Алтайский край</div>
<div class="bank-info">No.1326, Москва и обл.</div>
<div class="bank-

In [61]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text)) # очень крутые регулярные выражения - убирают все скобочки, тэги и прочую ерунду

In [62]:
cnt = 0
for cont in ratings_page.findAll('div', attrs = {'class':"b-bank-name"}):
    print(cnt)
    print(re.split('No.|,', html_stripper(cont))[:2])
    print("-------------------------")
    cnt+=1

0
['Абсолют Банк', '2306']
-------------------------
1
['Авангард', '2879']
-------------------------
2
['Аверс', '415']
-------------------------
3
['Автоградбанк', '1455']
-------------------------
4
['Агропромкредит', '2880']
-------------------------
5
['Агророс', '2860']
-------------------------
6
['Агросоюз', '1459']
-------------------------
7
['Азиатско-Тихоокеанский Банк', '1810']
-------------------------
8
['Ак Барс', '2590']
-------------------------
9
['Акибанк', '2587']
-------------------------
10
['Актив Банк', '2529']
-------------------------
11
['АктивКапитал Банк', '3006']
-------------------------
12
['Акцепт', '567']
-------------------------
13
['Александровский', '53']
-------------------------
14
['Алеф-Банк', '2119']
-------------------------
15
['Алмазэргиэнбанк', '2602']
-------------------------
16
['АлтайБизнес-Банк', '2388']
-------------------------
17
['Алтайкапиталбанк', '2659']
-------------------------
18
['Альфа-Банк', '1326']
---------------------

In [63]:
num_lics = []
bank_names = []

for cont in ratings_page.findAll('div', attrs = {'class':"b-bank-name"}):
    name, num = re.split('No.|,', html_stripper(cont))[:2]
    num_lics += [num]
    bank_names += [name]

num_lics = list(map(int,num_lics))

In [64]:
num_lics

[2306,
 2879,
 415,
 1455,
 2880,
 2860,
 1459,
 1810,
 2590,
 2587,
 2529,
 3006,
 567,
 53,
 2119,
 2602,
 2388,
 2659,
 1326,
 608,
 732,
 2929,
 101,
 2748,
 1067]

In [65]:
bank_names

['Абсолют Банк',
 'Авангард',
 'Аверс',
 'Автоградбанк',
 'Агропромкредит',
 'Агророс',
 'Агросоюз',
 'Азиатско-Тихоокеанский Банк',
 'Ак Барс',
 'Акибанк',
 'Актив Банк',
 'АктивКапитал Банк',
 'Акцепт',
 'Александровский',
 'Алеф-Банк',
 'Алмазэргиэнбанк',
 'АлтайБизнес-Банк',
 'Алтайкапиталбанк',
 'Альфа-Банк',
 'Аспект',
 'Ассоциация',
 'ББР Банк',
 'БКС — Инвестиционный Банк',
 'БМ-Банк (бывш. Банк Москвы)',
 'БайкалИнвестБанк']

In [66]:
def bank_names_and_lics(ratings_page):
    num_lics = []
    bank_names = []

    for cont in ratings_page.findAll('div', attrs = {'class':"b-bank-name"}):
        name, num = re.split('No.|,', html_stripper(cont))[:2]
        num_lics += [num]
        bank_names += [name]

    num_lics = list(map(int,num_lics))
    
    return (num_lics, bank_names)

In [ ]:
cnt=0
for cell in ratings_page.findAll('div', attrs = {"class":"b-table-ratings"}):
    cnt+=1
    print(cnt)
    #print(html_stripper(cell))
    print(cell)
    print("-------------------------------")

In [ ]:
ratings_page.findAll('div', attrs = {"class":"b-table-ratings"})

In [69]:
sp=re.split('<td>|</td>', str(ratings_page.findAll('div', attrs = {"class":"b-table-ratings"})).replace('\n', ' ').replace("<!--","").replace("-->","").replace('[',"").replace(']',""))

In [70]:
sp = list(map(html_stripper, sp))

In [71]:
sp = [el for el in sp if len(el.split())!= 0]

In [72]:
len(sp)

207

In [ ]:
for el in sp:
    print(el.split()[:4])

In [74]:
ratings  = pd.DataFrame()

In [75]:
s_MOODYS = ["P-1","P-2","P-3","NP","отозван"]

on_nat_MOODYS = ["Aaa.ru","Aa1.ru","Aa2.ru","Aa3.ru","A1.ru","A2.ru","A3.ru",\
                "Baa1.ru","Baa2.ru","Baa3.ru","Ba1.ru","Ba2.ru","Ba3.ru","B1.ru",\
                 "B2.ru","B3.ru","Caa1.ru","Caa2.ru","Caa3.ru","Ca.ru","C.ru",\
                    "отозван"]

fin_stab_MOODYS = ["A","B+","B","B-","C+","C","C-","D+","D","D-","E+","E","E-","отозван"]


In [ ]:
flag = 1
new_line = None
bank_ind = 0
num_lics = bank_names_and_lics(ratings_page)[0] 
bank_names = bank_names_and_lics(ratings_page)[1]
for i in range(len(sp)):
    l = sp[i].split()[:4]
    print(i,l)
    
    add_to_df = 0
    if flag:
        new_line = dict()
        flag=0
        
    if "Moody's" in l:
        if "Рус-рейтинг" in sp[i+1].split()[:4]:
            new_line["l_in_foreign_MOODYS"] = np.nan
            new_line["l_in_nat_MOODYS"] = np.nan
            new_line["MOODYS"] = np.nan
            new_line["s_in_foreign_MOODYS"] = np.nan
            new_line["s_in_nat_MOODYS"] = np.nan
            new_line["on_nat_MOODYS"] = np.nan
            new_line["fin_stab_MOODYS"] = np.nan
            new_line["dates_MOODYS"] = np.nan
        
        else:
            new_line["l_in_foreign_MOODYS"] = l[1]
            new_line["l_in_nat_MOODYS"] = l[2]
            new_line["MOODYS"] = l[3]
            
            if sp[i+1].split()[:2][0] in s_MOODYS:
                new_line["s_in_foreign_MOODYS"] = sp[i+1].split()[:4][0]
            
            if sp[i+1].split()[:2][1] in s_MOODYS:
                new_line["s_in_nat_MOODYS"] = sp[i+1].split()[:4][1]
            
            if sp[i+2].split()[:4][0] in on_nat_MOODYS:
                if len(sp[i+3].split()[:4][0]) == 10:
                    new_line["on_nat_MOODYS"] = np.nan
                else:
                    new_line["on_nat_MOODYS"] = sp[i+2].split()[:4][0]
                    
            if sp[i+2].split()[:4][0] in fin_stab_MOODYS: 
                new_line["fin_stab_MOODYS"] = sp[i+2].split()[:4][0]
                
            elif len(sp[i+2].split()[:4][0]) == 10:
                new_line["dates_MOODYS"] = sp[i+2].split()[:4][0]
                
            if sp[i+3].split()[:4][0] in fin_stab_MOODYS: 
                new_line["fin_stab_MOODYS"] = sp[i+3].split()[:4][0]
                
            elif len(sp[i+3].split()[:4][0]) == 10:
                new_line["dates_MOODYS"] = sp[i+3].split()[:4][0]
            
            if len(sp[i+4].split()[:4][0]) == 10:
                new_line["dates_MOODYS"] = sp[i+4].split()[:4][0]
            
            
    if "Эксперт" in l:
        if "НРА" in sp[i+1].split()[:4]:
            new_line["EXPERT_RA"] = np.nan
            new_line["EXPERT_RA_prognosis"] = np.nan
            new_line["EXPERT_RA_dates"] = np.nan
        else:
            new_line["EXPERT_RA"] = sp[i+1].split()[:4][0]
            if new_line["EXPERT_RA"] == sp[i+1].split()[:4][1]:
                new_line["EXPERT_RA_prognosis"] = np.nan
            else:
                new_line["EXPERT_RA_prognosis"] = sp[i+1].split()[:4][1]
            
            new_line["EXPERT_RA_dates"] = sp[i+2].split()[:4][0]
            
    if "НРА" in l:
        if 'AK&M' in sp[i+1].split()[:4]:
            new_line["NRA"] = np.nan
            new_line["NRA_prognosis"] = np.nan
            new_line["NRA_dates"] = np.nan
        else:
            new_line["NRA"] = sp[i+1].split()[:4][0]
            if new_line["NRA"] == sp[i+1].split()[:4][1]:
                new_line["NRA_prognosis"]= np.nan
            else:
                new_line["NRA_prognosis"] = sp[i+1].split()[:4][1]
                
            new_line["NRA_dates"] = sp[i+2].split()[:4][0]
            
    if "AK&M" in l:
        try:
            if "Moody's" in sp[i+1].split()[:4]:
                new_line["AK&M"] = np.nan
                new_line["AK&M_prognosis"] = np.nan
                new_line["AK&M_dates"] = np.nan
            
            else:
                new_line["AK&M"] = sp[i+1].split()[:4][0]
                if new_line["AK&M"] == sp[i+1].split()[:4][1]:
                    new_line["AK&M_prognosis"]= np.nan
                else:
                    new_line["AK&M_prognosis"] = sp[i+1].split()[:4][1]
                
                new_line["AK&M_dates"] = sp[i+2].split()[:4][0]
        except:
            new_line["AK&M"] = np.nan
            new_line["AK&M_prognosis"] = np.nan
            new_line["AK&M_dates"] = np.nan    
        
        flag = 1
        
    if flag:
        new_line["num_of_lic"] = num_lics[bank_ind]
        new_line["bank_name"] = bank_names[bank_ind]

        bank_ind += 1
        ratings = ratings.append(new_line, ignore_index=True)
        flag = 0


In [77]:
ratings = ratings[["num_of_lic","bank_name","l_in_foreign_MOODYS","l_in_nat_MOODYS", "MOODYS", "s_in_foreign_MOODYS", "s_in_nat_MOODYS", "on_nat_MOODYS", "fin_stab_MOODYS", "dates_MOODYS", "EXPERT_RA", "EXPERT_RA_prognosis","EXPERT_RA_dates","NRA","NRA_prognosis", "NRA_dates","AK&M","AK&M_prognosis","AK&M_dates"]]
ratings.head()

num_of_lic       bank_name l_in_foreign_MOODYS l_in_nat_MOODYS      MOODYS  \
0      2306.0    Абсолют Банк                  B1              B1  негативный   
1      2879.0        Авангард                  B2              B2  стабильный   
2       415.0           Аверс                 NaN             NaN         NaN   
3      1455.0    Автоградбанк                 NaN             NaN         NaN   
4      2880.0  Агропромкредит                  B2              B2  негативный   

  s_in_foreign_MOODYS s_in_nat_MOODYS on_nat_MOODYS fin_stab_MOODYS  \
0                  NP              NP       отозван              E+   
1                  NP              NP       отозван              E+   
2                 NaN             NaN           NaN             NaN   
3                 NaN             NaN           NaN             NaN   
4                  NP              NP           NaN              E+   

  dates_MOODYS EXPERT_RA EXPERT_RA_prognosis EXPERT_RA_dates  NRA  \
0   26.04.2016     ruBBB          стабильный      28.04.2017  NaN   
1   18.03.2016       NaN                 NaN             NaN  NaN   
2          NaN      ruA+          стабильный      28.04.2017  NaN   
3          NaN     ruBB-          стабильный      27.04.2017  NaN   
4   21.04.2017       NaN                 NaN             NaN  NaN   

  NRA_prognosis NRA_dates AK&M AK&M_prognosis AK&M_dates  
0           NaN       NaN  NaN            NaN        NaN  
1           NaN       NaN  NaN            NaN        NaN  
2           NaN       NaN  NaN            NaN        NaN  
3           NaN       NaN  NaN            NaN        NaN  
4           NaN       NaN  NaN            NaN        NaN

# Вишенка на тортике. Функция, делающая всё:) / The func, which does all necessary.

In [78]:
def bankiru_parsing(pages_number: int, verbose = False):
    """
    return: DataFrame table corresponding banki.ru table
    """
    main_url = "http://www.banki.ru/banks/ratings/agency/?PAGEN_2={}&mode=2"
    
    ratings = pd.DataFrame()
    
    #unique moodys values, just lists of constants 
    s_MOODYS = ["P-1","P-2","P-3","NP","отозван"]

    on_nat_MOODYS = ["Aaa.ru","Aa1.ru","Aa2.ru","Aa3.ru","A1.ru","A2.ru","A3.ru",\
                "Baa1.ru","Baa2.ru","Baa3.ru","Ba1.ru","Ba2.ru","Ba3.ru","B1.ru",\
                 "B2.ru","B3.ru","Caa1.ru","Caa2.ru","Caa3.ru","Ca.ru","C.ru",\
                    "отозван"]

    fin_stab_MOODYS = ["A","B+","B","B-","C+","C","C-","D+","D","D-","E+","E","E-","отозван"]

    for page in range(1, pages_number+1):
        
        if verbose:
            print(page)
        ratings_page = requests.get(main_url.format(page))
        ratings_page = ratings_page.content
        ratings_page = BeautifulSoup(ratings_page, "lxml")
    
        sp=re.split('<td>|</td>', str(ratings_page.findAll('div', attrs = {"class":"b-table-ratings"})).replace('\n', ' ').replace("<!--","").replace("-->","").replace('[',"").replace(']',""))
        sp = list(map(html_stripper, sp))
        sp = [el for el in sp if len(el.split())!= 0]
    
        
        flag = 1
        new_line = None
        bank_ind = 0
        num_lics = bank_names_and_lics(ratings_page)[0] 
        bank_names = bank_names_and_lics(ratings_page)[1]
        for i in range(len(sp)):    
            l = sp[i].split()[:4]
            if flag:
                new_line = dict()
                flag=0
        
            if "Moody's" in l:
                if "Рус-рейтинг" in sp[i+1].split()[:4]:
                    new_line["l_in_foreign_MOODYS"] = np.nan
                    new_line["l_in_nat_MOODYS"] = np.nan
                    new_line["MOODYS"] = np.nan
                    new_line["s_in_foreign_MOODYS"] = np.nan
                    new_line["s_in_nat_MOODYS"] = np.nan
                    new_line["on_nat_MOODYS"] = np.nan
                    new_line["fin_stab_MOODYS"] = np.nan
                    new_line["dates_MOODYS"] = np.nan
                elif len(l) == 1: #special code for 2312 num lic
                    new_line["l_in_foreign_MOODYS"] = np.nan
                    new_line["l_in_nat_MOODYS"] = np.nan
                    new_line["MOODYS"] = np.nan
                    new_line["s_in_foreign_MOODYS"] = np.nan
                    new_line["s_in_nat_MOODYS"] = np.nan
                    if sp[i+1].split()[:4][0] in on_nat_MOODYS:
                        if len(sp[i+1].split()[:4][0]) == 10:
                            new_line["on_nat_MOODYS"] = np.nan
                        else:
                            new_line["on_nat_MOODYS"] = sp[i+1].split()[:4][0]
                    
                    new_line["fin_stab_MOODYS"] = np.nan

                    if len(sp[i+2].split()[:4][0]) == 10:
                        new_line["dates_MOODYS"] = sp[i+2].split()[:4][0]
                else:
                    
                    new_line["l_in_foreign_MOODYS"] = l[1]
                    new_line["l_in_nat_MOODYS"] = l[2]
                    new_line["MOODYS"] = l[3]

                    if sp[i+1].split()[:2][0] in s_MOODYS:
                        new_line["s_in_foreign_MOODYS"] = sp[i+1].split()[:4][0]
            
                    if sp[i+1].split()[:2][1] in s_MOODYS:
                        new_line["s_in_nat_MOODYS"] = sp[i+1].split()[:4][1]
            
                    if sp[i+2].split()[:4][0] in on_nat_MOODYS:
                        if len(sp[i+3].split()[:4][0]) == 10:
                            new_line["on_nat_MOODYS"] = np.nan
                        else:
                            new_line["on_nat_MOODYS"] = sp[i+2].split()[:4][0]
                    
                    if sp[i+2].split()[:4][0] in fin_stab_MOODYS: 
                        new_line["fin_stab_MOODYS"] = sp[i+2].split()[:4][0]
                
                    elif len(sp[i+2].split()[:4][0]) == 10:
                        new_line["dates_MOODYS"] = sp[i+2].split()[:4][0]
                
                    if sp[i+3].split()[:4][0] in fin_stab_MOODYS: 
                        new_line["fin_stab_MOODYS"] = sp[i+3].split()[:4][0]
                
                    elif len(sp[i+3].split()[:4][0]) == 10:
                        new_line["dates_MOODYS"] = sp[i+3].split()[:4][0]
            
                    if len(sp[i+4].split()[:4][0]) == 10:
                        new_line["dates_MOODYS"] = sp[i+4].split()[:4][0]
                        
            if "Эксперт" in l:
                if "НРА" in sp[i+1].split()[:4]:
                    new_line["EXPERT_RA"] = np.nan
                    new_line["EXPERT_RA_prognosis"] = np.nan
                    new_line["EXPERT_RA_dates"] = np.nan
                else:
                    new_line["EXPERT_RA"] = sp[i+1].split()[:4][0]
                    if new_line["EXPERT_RA"] == sp[i+1].split()[:4][1]:
                        new_line["EXPERT_RA_prognosis"] = np.nan
                    else:
                        new_line["EXPERT_RA_prognosis"] = sp[i+1].split()[:4][1]
            
                    new_line["EXPERT_RA_dates"] = sp[i+2].split()[:4][0]
            
            if "НРА" in l:
                if 'AK&M' in sp[i+1].split()[:4]:
                    new_line["NRA"] = np.nan
                    new_line["NRA_prognosis"] = np.nan
                    new_line["NRA_dates"] = np.nan
                else:
                    new_line["NRA"] = sp[i+1].split()[:4][0]
                    if new_line["NRA"] == sp[i+1].split()[:4][1]:
                        new_line["NRA_prognosis"]= np.nan
                    else:
                        new_line["NRA_prognosis"] = sp[i+1].split()[:4][1]
                
                    new_line["NRA_dates"] = sp[i+2].split()[:4][0]
            
            if "AK&M" in l:
                try:
                    if "Moody's" in sp[i+1].split()[:4]:
                        new_line["AK&M"] = np.nan
                        new_line["AK&M_prognosis"] = np.nan
                        new_line["AK&M_dates"] = np.nan
            
                    else:
                        new_line["AK&M"] = sp[i+1].split()[:4][0]
                        if new_line["AK&M"] == sp[i+1].split()[:4][1]:
                            new_line["AK&M_prognosis"]= np.nan
                        else:
                            new_line["AK&M_prognosis"] = sp[i+1].split()[:4][1]
                
                        new_line["AK&M_dates"] = sp[i+2].split()[:4][0]
                except:
                    new_line["AK&M"] = np.nan
                    new_line["AK&M_prognosis"] = np.nan
                    new_line["AK&M_dates"] = np.nan    
        
                flag = 1
        
            if flag:
                new_line["num_of_lic"] = num_lics[bank_ind]
                new_line["bank_name"] = bank_names[bank_ind]
                ratings = ratings.append(new_line, ignore_index=True)
                bank_ind += 1
                flag = 0 

    ratings = ratings[["num_of_lic", "bank_name","l_in_foreign_MOODYS","l_in_nat_MOODYS", "MOODYS", "s_in_foreign_MOODYS", "s_in_nat_MOODYS", "on_nat_MOODYS", "fin_stab_MOODYS", "dates_MOODYS", "EXPERT_RA", "EXPERT_RA_prognosis","EXPERT_RA_dates","NRA","NRA_prognosis", "NRA_dates","AK&M","AK&M_prognosis","AK&M_dates"]]

    return ratings

### На сайте banki.ru есть 11 страниц таблицы с рейтингами о банках / In banki.ru there are 11 table pages with ratings of banks.

In [79]:

data=bankiru_parsing(pages_number = 11)

In [80]:
data

num_of_lic                                bank_name l_in_foreign_MOODYS  \
0        2306.0                             Абсолют Банк                  B1   
1        2879.0                                 Авангард                  B2   
2         415.0                                    Аверс                 NaN   
3        1455.0                             Автоградбанк                 NaN   
4        2880.0                           Агропромкредит                  B2   
5        2860.0                                  Агророс                 NaN   
6        1459.0                                 Агросоюз                 NaN   
7        1810.0              Азиатско-Тихоокеанский Банк             отозван   
8        2590.0                                  Ак Барс                  B2   
9        2587.0                                  Акибанк                  B3   
10       2529.0                               Актив Банк                 NaN   
11       3006.0                        АктивКапитал Банк                 NaN   
12        567.0                                   Акцепт                 NaN   
13         53.0                          Александровский                 NaN   
14       2119.0                                Алеф-Банк                 NaN   
15       2602.0                          Алмазэргиэнбанк                 NaN   
16       2388.0                         АлтайБизнес-Банк                 NaN   
17       2659.0                         Алтайкапиталбанк                 NaN   
18       1326.0                               Альфа-Банк                 Ba2   
19        608.0                                   Аспект                 NaN   
20        732.0                               Ассоциация                 NaN   
21       2929.0                                 ББР Банк                 NaN   
22        101.0                БКС — Инвестиционный Банк                 NaN   
23       2748.0              БМ-Банк (бывш. Банк Москвы)             отозван   
24       1067.0                         БайкалИнвестБанк                 NaN   
25        128.0                          Балтийский Банк                 NaN   
26       3176.0                           Балтинвестбанк                Caa1   
27        436.0                   Банк «Санкт-Петербург»                  B1   
28        558.0                         Банк «Советский»                 NaN   
29       2209.0                       Банк «ФК Открытие»                  B2   
..          ...                                      ...                 ...   
227      2997.0                              Углеметбанк                 NaN   
228       249.0                                  Урал ФД                  B3   
229      2519.0                          Уралкапиталбанк                 NaN   
230       153.0                           Уралприватбанк                 NaN   
231      2964.0                             Уралпромбанк                 NaN   
232       429.0  Уральский Банк Реконструкции и Развития                 NaN   
233       596.0                                   Уссури                 NaN   
234      2063.0                                  ФорБанк                 NaN   
235      1885.0                                Фора-Банк                 NaN   
236      2208.0                                 Форштадт                 NaN   
237      1049.0             Хакасский Муниципальный Банк                 NaN   
238       254.0                                   Хлынов                 NaN   
239       503.0                                   Холмск                 NaN   
240       316.0                         Хоум Кредит Банк             отозван   
241      2225.0                             Центр-инвест                 Ba3   
242      2670.0              Центрально-Европейский Банк                 NaN   
243       485.0                               Челиндбанк                 NaN   
244       493.0                          Челябинвестбанк                 NaN   
245  

In [81]:
data.to_csv("bank_ratings.csv",index=False)

# Преобразование значений признаков к числовому формату / Transformation features to numeric format
(see **расшифровка-признаков-в-рейтингах-банков.docx** for understanding)

In [82]:
data.dtypes

num_of_lic             float64
bank_name               object
l_in_foreign_MOODYS     object
l_in_nat_MOODYS         object
MOODYS                  object
s_in_foreign_MOODYS     object
s_in_nat_MOODYS         object
on_nat_MOODYS           object
fin_stab_MOODYS         object
dates_MOODYS            object
EXPERT_RA               object
EXPERT_RA_prognosis     object
EXPERT_RA_dates         object
NRA                     object
NRA_prognosis           object
NRA_dates               object
AK&M                    object
AK&M_prognosis          object
AK&M_dates              object
dtype: object

In [83]:
data["num_of_lic"] = data["num_of_lic"].astype("str").astype(float).astype(int) 

### Preprocessing of "l_in_foreign_MOODYS"

In [84]:
data["l_in_foreign_MOODYS"] = data["l_in_foreign_MOODYS"].astype("str")
data["l_in_foreign_MOODYS"].replace(to_replace = ["C","Ca","Caa3","Caa2","Caa1","B3","B2","B1", "Ba3", "Ba2", "Ba1", "Baa3","Baa2", "Baa1", "A3","A2","A1", "Aa3", "Aa2", "Aa1", "Aaa", "nan"], value = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,np.nan], inplace = True)

#Values of Series are set in numeric except value "nan", which will be NaN
data["l_in_foreign_MOODYS"] = pd.to_numeric(data["l_in_foreign_MOODYS"], errors="coerce")

### Preprocessing of "l_in_nat_MOODYS"

In [85]:
data["l_in_nat_MOODYS"] = data["l_in_nat_MOODYS"].astype("str")
data["l_in_nat_MOODYS"].replace(to_replace = ["C","Ca","Caa3","Caa2","Caa1","B3","B2","B1", "Ba3", "Ba2", "Ba1", "Baa3","Baa2", "Baa1", "A3","A2","A1", "Aa3", "Aa2", "Aa1", "Aaa", "nan"], value = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,np.nan], inplace = True)

#Values of Series are set in numeric except value "nan", which will be NaN
data["l_in_nat_MOODYS"] = pd.to_numeric(data["l_in_nat_MOODYS"], errors="coerce")

### Preprocessing of "MOODYS"

In [86]:
data["MOODYS"] = data["MOODYS"].astype("str")
data["MOODYS"].replace(to_replace = ['негативный', 'стабильный', 'позитивный'], value = [0,1,2], inplace = True)

#Values of Series are set in numeric except value nan, which will be NaN
data["MOODYS"] = pd.to_numeric(data["MOODYS"], errors="coerce")

### Preprocessing of "s_in_foreign_MOODYS"

In [100]:
data["s_in_foreign_MOODYS"].replace(to_replace = ['NP','P-3','P-2','P-1',"отозван"],\
                                    value = [0,1,2,3,-1], inplace = True)

### Preprocessing of "s_in_nat_MOODYS"

In [101]:
data["s_in_nat_MOODYS"].replace(to_replace = ['NP','P-3','P-2','P-1',"отозван"],\
                                value = [0,1,2,3,-1], inplace = True)

### Preprocessing of "on_nat_MOODYS"

In [ ]:
data["on_nat_MOODYS"] = data["on_nat_MOODYS"].astype("str")
#Values of Series are set in numeric except "nan" and trully string vars, which will be NaN
data["on_nat_MOODYS"] = pd.to_numeric(data["on_nat_MOODYS"], errors="coerce")

### Preprocessing of "fin_stab_MOODYS"


In [89]:
data["fin_stab_MOODYS"] = data["fin_stab_MOODYS"].astype("str")
data["fin_stab_MOODYS"].replace(to_replace = ['E-', 'E', 'E+', "D-","D","D+", "C-",\
            "C", "C+", "B-", "B", "B+", "A"],value = [0,1,2,3,4,5,6,7,8,9,10,11,12], inplace = True)
#Values of Series are set in numeric except value "nan", which will be NaN
data["fin_stab_MOODYS"] = pd.to_numeric(data["fin_stab_MOODYS"], errors="coerce")

### Preprocessing of "EXPERT_RA"

In [90]:
data["EXPERT_RA"] = data["EXPERT_RA"].astype("str")
data["EXPERT_RA"].replace(to_replace = "-2", value = '-1', inplace = True)
#providing same format
data["EXPERT_RA"].replace(to_replace = ['D', 'RD', "C", "CC", "CCC","B-", "B", \
        "B+", "BB-", "BB", "BB+", "BBB-", "BBB", "BBB+", "A-", "A", "A+", "AA-",\
        "AA", "AA+", "AAA", "C++", "B++", "A++"],\
        value = ['ruD', 'ruRD', "ruC", "ruCC", "ruCCC","ruB-", "ruB", "ruB+", \
                 "ruBB-", "ruBB", "ruBB+", "ruBBB-", "ruBBB", "ruBBB+", "ruA-",\
                 "ruA", "ruA+", "ruAA-", "ruAA", "ruAA+", "ruAAA", "ruCCC", "ruBB-", "ruAA-"], inplace = True)
data["EXPERT_RA"].replace(to_replace = ['ruD', 'ruRD', "ruC", "ruCC", "ruCCC","ruB-", "ruB", "ruB+", "ruBB-",\
        "ruBB", "ruBB+", "ruBBB-", "ruBBB", "ruBBB+", "ruA-", "ruA", "ruA+", "ruAA-", "ruAA", "ruAA+", "ruAAA"],\
        value = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], inplace = True)

#Values of Series are set in numeric except value "nan", which will be NaN
data["EXPERT_RA"] = pd.to_numeric(data["EXPERT_RA"], errors="coerce")

### Preprocessing of "EXPERT_RA_prognosis"

In [91]:
data["EXPERT_RA_prognosis"] = data["EXPERT_RA_prognosis"].astype("str")
data["EXPERT_RA_prognosis"].replace(to_replace = ['негативный', 'стабильный', 'позитивный'], value = [0,1,2], inplace = True)

#Values of Series are set in numeric except value nan, which will be nan
data["EXPERT_RA_prognosis"] = pd.to_numeric(data["EXPERT_RA_prognosis"], errors="coerce")

### Preprocessing of "EXPERT_RA_prognosis"


In [92]:
data["NRA"] = data["NRA"].astype("str")
data["NRA"].replace(to_replace = "-2", value = '-1', inplace = True)
data["NRA"].replace(to_replace = ['D', 'C-', "C", "C+","CC-", "CC", "CC+", "B-", "B", "B+", "BB-", \
        "BB", "BB+", "BBB-", "BBB", "BBB+", "A-", "A", "A+", "AA-", "AA", "AA+", "AAA"],\
                    value = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22], inplace = True)
#Values of Series are set in numeric except value "nan", which will be NaN
data["NRA"] = pd.to_numeric(data["NRA"], errors="coerce")

### Preprocessing of "NRA_prognosis"


In [93]:
data["NRA_prognosis"] = data["NRA_prognosis"].astype("str")
data["NRA_prognosis"].replace(to_replace = ['негативный', 'стабильный', 'позитивный'],\
                              value = [0,1,2], inplace = True)

#Values of Series are set in numeric except value nan, which will be nan
data["NRA_prognosis"] = pd.to_numeric(data["NRA_prognosis"], errors="coerce")

### Preprocessing of "AK&M"

In [94]:
data["AK&M"] = data["AK&M"].astype("str")
data["AK&M"].replace(to_replace = "-2", value = '-1', inplace = True)
data["AK&M"].replace(to_replace = ['D', 'C', "C+","C++", "B", "B+", "B++", "A", "A+", "A++"],\
                     value = [0,1,2,3,4,5,6,7,8,9], inplace = True)
#Values of Series are set in numeric except value "nan", which will be NaN
data["AK&M"] = pd.to_numeric(data["AK&M"], errors="coerce")

### Preprocessing of "AK&M_prognosis"

In [95]:
data["AK&M_prognosis"] = data["AK&M_prognosis"].astype("str")
data["AK&M_prognosis"].replace(to_replace = ['негативный', 'стабильный', 'позитивный'],\
                               value = [0,1,2], inplace = True)

#Values of Series are set in numeric except value nan, which will be nan
data["AK&M_prognosis"] = pd.to_numeric(data["AK&M_prognosis"], errors="coerce")

In [102]:
data.head()

num_of_lic       bank_name  l_in_foreign_MOODYS  l_in_nat_MOODYS  MOODYS  \
0        2306    Абсолют Банк                  7.0              7.0     0.0   
1        2879        Авангард                  6.0              6.0     1.0   
2         415           Аверс                  NaN              NaN     NaN   
3        1455    Автоградбанк                  NaN              NaN     NaN   
4        2880  Агропромкредит                  6.0              6.0     0.0   

   s_in_foreign_MOODYS  s_in_nat_MOODYS  on_nat_MOODYS  fin_stab_MOODYS  \
0                  0.0              0.0            NaN              2.0   
1                  0.0              0.0            NaN              2.0   
2                  NaN              NaN            NaN              NaN   
3                  NaN              NaN            NaN              NaN   
4                  0.0              0.0            NaN              2.0   

  dates_MOODYS  EXPERT_RA  EXPERT_RA_prognosis EXPERT_RA_dates  NRA  \
0   26.04.2016       12.0                  1.0      28.04.2017  NaN   
1   18.03.2016        NaN                  NaN             NaN  NaN   
2          NaN       16.0                  1.0      28.04.2017  NaN   
3          NaN        8.0                  1.0      27.04.2017  NaN   
4   21.04.2017        NaN                  NaN             NaN  NaN   

   NRA_prognosis NRA_dates  AK&M  AK&M_prognosis AK&M_dates  
0            NaN       NaN   NaN             NaN        NaN  
1            NaN       NaN   NaN             NaN        NaN  
2            NaN       NaN   NaN             NaN        NaN  
3            NaN       NaN   NaN             NaN        NaN  
4            NaN       NaN   NaN             NaN        NaN

In [103]:
data.tail()

num_of_lic        bank_name  l_in_foreign_MOODYS  l_in_nat_MOODYS  \
252          52    Энергомашбанк                  NaN              NaN   
253        1307  Энерготрансбанк                  NaN              NaN   
254        2772    Юг-Инвестбанк                  NaN              NaN   
255        3467         Юнистрим                  NaN              NaN   
256        3185          ЯР-Банк                  NaN              NaN   

     MOODYS  s_in_foreign_MOODYS  s_in_nat_MOODYS  on_nat_MOODYS  \
252     NaN                  NaN              NaN            NaN   
253     NaN                  NaN              NaN            NaN   
254     NaN                  NaN              NaN            NaN   
255     NaN                  NaN              NaN            NaN   
256     NaN                  NaN              NaN            NaN   

     fin_stab_MOODYS dates_MOODYS  EXPERT_RA  EXPERT_RA_prognosis  \
252              NaN          NaN        NaN                  NaN   
253              NaN          NaN       13.0                  1.0   
254              NaN          NaN       11.0                  1.0   
255              NaN          NaN       13.0                  1.0   
256              NaN          NaN        NaN                  NaN   

    EXPERT_RA_dates  NRA  NRA_prognosis   NRA_dates  AK&M  AK&M_prognosis  \
252      30.05.2016  NaN            NaN         NaN   NaN             NaN   
253      16.05.2017  NaN            NaN         NaN   NaN             NaN   
254      26.05.2017  NaN            NaN         NaN   NaN             NaN   
255      24.05.2017  NaN            NaN         NaN   NaN             NaN   
256             NaN  NaN            NaN  25.11.2016   NaN             NaN   

    AK&M_dates  
252        NaN  
253        NaN  
254        NaN  
255        NaN  
256        NaN

In [104]:
data.to_csv("preprocessed_bank_ratings.csv", index=False)